In [ ]:
%load_ext autoreload
%autoreload 2
# import seaborn.apionly as sns
# seaborn.reset_orig()
# mpl.rcParams.update(inline_rc)
import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import skimage
import skimage.io

%matplotlib inline
inline_rc = dict(mpl.rcParams)

from nima import nima

cm = plt.cm.inferno_r

fp = "../../tests/data/1b_c16_15.tif"

# usage

In [ ]:
channels = ["G", "R", "C"]
# dark = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/dark/dark-25_500.tif')
# flat = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/flat/flat-C-dark-37bis_500.tif')

d_im, n_ch, times = nima.read_tiff(fp, channels)

In [ ]:
d_im_bg, bgs, ff, bgv = nima.d_bg(d_im, downscale=(2, 2), kind="li_adaptive")
bgs

In [ ]:
d_im_bg["G"].__sizeof__()

In [ ]:
scipy.stats.distributions.norm.fit(bgv["C"][1])

In [ ]:
import seaborn as sns

sns.distplot(bgv["C"][1], fit=scipy.stats.distributions.norm)
plt.yscale("log")

In [ ]:
ff["G"][1][0]

In [ ]:
for i in range(4):
    plt.hist(bgv["C"][i], histtype="step", bins=32, log=1)

In [ ]:
%%timeit
nima.d_mask_label(
    d_im_bg, threshold_method="yen", min_size=2000, channels=channels, watershed=0
)

In [ ]:
d_im.keys()

In [ ]:
f = nima.d_show(d_im_bg)

In [ ]:
%%time
meas, pr = nima.d_meas_props(d_im_bg)

In [ ]:
f = nima.d_plot_meas(bgs, meas, channels=channels)

In [ ]:
plt.subplot(1, 2, 1)
skimage.io.imshow(d_im_bg["r_cl"][2], vmin=-0.0, vmax=1.1)
plt.subplot(1, 2, 2)
skimage.io.imshow(d_im_bg["r_pH"][2], vmin=7.3, vmax=10.0)

## background, AF, target cells

In [ ]:
channels3 = ["G", "R", "C", "G2"]
dim3, _, _ = nima.read_tiff(
    "/home/dati/GBM_persson/data/15.02.05_cal-GBM5-pBJclop/ph633/1_20_40.tif", channels3
)
dim3, bgm3, _, bgv3 = nima.d_bg(dim3, downscale=(2, 2), kind="li_adaptive")
c = dim3["C"]
g = dim3["G"]
r = dim3["R"]

bc = bgv3["C"][2]
bg = bgv3["G"][3]
br = bgv3["R"][3]

c = d_im_bg['C']
g = d_im_bg['G']
r = d_im_bg['R']

bc = bgv['C'][2]
bg = bgv['G'][3]
br = bgv['R'][3]

In [ ]:
mmin = min(len(bc), len(bg), len(br))

print("c =", len(bc))
print("g =", len(bg))
print("r =", len(br))

plt.figure(figsize=(7.5, 6))

plt.subplot(2, 2, 1)
mmin = min(len(bc), len(bg))
plt.hexbin(bg[:mmin], bc[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(2, 2, 2)
mmin = min(len(bc), len(br))
plt.hexbin(br[:mmin], bc[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(2, 2, 4)
mmin = min(len(br), len(bg))
plt.hexbin(br[:mmin], bg[:mmin], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.hexbin(g.ravel(), c.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(2, 2, 2)
plt.hexbin(r.ravel(), c.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(2, 2, 4)
plt.hexbin(r.ravel(), g.ravel(), bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

axins = plt.axes([0.2, 0.12, 0.28, 0.28])
axins.hexbin(
    r.ravel(), g.ravel(), extent=(0, 80, 0, 150), bins="log", cmap=plt.cm.viridis_r
)

mark_inset(ax, axins, loc1=1, loc2=4, fc="none", ec="0.5")

In [ ]:
bf = skimage.io.imread(
    "/home/dati/GBM_persson/data/15.02.05_cal-GBM5-pBJclop/ph633/1_20_39.tif"
)
bf = skimage.exposure.adjust_log(bf)

xini = 1090
xfin = 1150
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.xlabel("Green")
plt.ylabel("Cyan")
plt.hexbin(
    r[:, 200:500, xini:xfin].ravel(),
    c[:, 200:500, xini:xfin].ravel(),
    bins="log",
    cmap=plt.cm.viridis_r,
)
cb = plt.colorbar()
plt.subplot(4, 2, 2)
plt.imshow(bf[1, 200:500, xini:xfin], vmin=50, vmax=550, cmap=plt.cm.Greys)
plt.subplot(4, 2, 4)
plt.imshow(c[1, 200:500, xini:xfin], cmap=plt.cm.Blues_r)

plt.subplot(4, 2, 6)
plt.imshow(g[1, 200:500, xini:xfin], cmap=plt.cm.Greens_r)

plt.subplot(4, 2, 8)
plt.imshow(r[1, 200:500, xini:xfin], cmap=plt.cm.Reds_r)

plt.subplots_adjust()

In [ ]:
bf = skimage.io.imread(
    "/home/dati/GBM_persson/data/15.02.05_cal-GBM5-pBJclop/ph633/1_20_39.tif"
)
bf = skimage.exposure.adjust_log(bf)

xini = 1150
xfin = 1388
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.xlabel("Green")
plt.ylabel("Cyan")
plt.hexbin(
    r[:, 200:500, xini:xfin].ravel(),
    c[:, 200:500, xini:xfin].ravel(),
    bins="log",
    cmap=plt.cm.viridis_r,
)
cb = plt.colorbar()
plt.subplot(4, 2, 2)
plt.imshow(bf[1, 200:500, xini:xfin], vmin=50, vmax=550, cmap=plt.cm.Greys)
plt.subplot(4, 2, 4)
plt.imshow(c[1, 200:500, xini:xfin], cmap=plt.cm.Blues_r)

plt.subplot(4, 2, 6)
plt.imshow(g[1, 200:500, xini:xfin], cmap=plt.cm.Greens_r)

plt.subplot(4, 2, 8)
plt.imshow(r[1, 200:500, xini:xfin], cmap=plt.cm.Reds_r)

plt.subplots_adjust()

In [ ]:
import statsmodels.api as sm


# import pylab
def qqplot(v1, v2):
    ml = min(len(v1), len(v2))
    sm.qqplot_2samples(v1[:ml], v2[:ml])
    # pylab.show()


qqplot(bgv3["R"][2], bgv3["G"][2])
qqplot(bgv3["R"][1], bgv3["R"][2])

# flat image correction

In [ ]:
g1a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c1_2.tf8", channels)[0][
    "G"
][2]
g2a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c2_2.tf8", channels)[0][
    "G"
][2]
g3a = nima.read_tiff("/home/dati/dt-clop3/data/210917/3alpha/c3_2.tf8", channels)[0][
    "G"
][2]
g1 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c1_2.tf8", channels)[0]["G"][2]
g2 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c2_2.tf8", channels)[0]["G"][2]
g3 = nima.read_tiff("/home/dati/dt-clop3/data/210917/3/c3_2.tf8", channels)[0]["G"][2]

In [ ]:
plt.subplot(1, 2, 1)
skimage.io.imshow(
    skimage.filters.gaussian(np.max([g1, g2, g3, g1a, g2a, g3a], axis=0), sigma=500)
)
plt.subplot(1, 2, 2)
skimage.io.imshow(skimage.filters.gaussian(np.max([g1, g2, g3], axis=0), sigma=500))

# memmap

In [ ]:
import tifffile

skimage.io.imshow(
    tifffile.memmap("/home/dati/dt-clop3/data/210917/3/c1_2.tf8", mode="r", page=26)
)

In [ ]:
with tifffile.TiffFile("/home/dati/dt-clop3/data/210917/3alpha/c1_2.tf8") as tif:
    im = tif.asarray()
    axes = tif.series[0].axes
    print(axes)

# zarr + dask

In [ ]:
from dask import distributed

client = distributed.Client()
print(client.dashboard_link)

In [ ]:
import dask.array as da

import numpy as np
import matplotlib.pyplot as plt
import skimage
import skimage.io
import skimage.filters
import zarr
import tifffile

store = tifffile.imread("/home/dati/dt-clop3/data/210920/flatxy.tf8", aszarr=True)

zc1a = zarr.open(store)
zc1a.info

In [ ]:
store

In [ ]:
store.close()

In [ ]:
dd = da.from_zarr(store)

In [ ]:
dd.shape

In [ ]:
dd = da.from_array(zc1a)

In [ ]:
pj = dd[:, 0].max()

In [ ]:
pj

In [ ]:
pj.compute()

In [ ]:
mp = np.max(zc1a, axis=0)

In [ ]:
plt.subplot(2, 2, 1)
skimage.io.imshow(mp[1])
plt.subplot(2, 2, 2)
skimage.io.imshow(skimage.filters.gaussian(mp[0], sigma=500))
plt.subplot(2, 2, 3)
skimage.io.imshow(skimage.filters.gaussian(mp[1], sigma=500))
plt.subplot(2, 2, 4)
skimage.io.imshow(skimage.filters.gaussian(mp[2], sigma=500))

In [ ]:
(np.maximum.reduce(zc1a) == mp).all()

In [ ]:
zc1a.set_basic_selection

In [ ]:
from dask.diagnostics import ProgressBar
from skimage import filters


def flat(fp):
    store = tifffile.imread(fp, aszarr=True)
    dask_array = da.from_zarr(store)
    mpd = np.max(
        dask_array[:-1], axis=0
    )  # OME series expected vs got frames for interrupted acquisition
    # mpd = np.mean(dask_array[:-1], axis=0)
    # mpd = np.maximum.reduce(dask_array[:-1])
    with ProgressBar():
        mp = mpd.compute()
    plt.subplot(2, 2, 1)
    skimage.io.imshow(mp[1])
    plt.subplot(2, 2, 2)
    skimage.io.imshow(skimage.filters.gaussian(mp[0], sigma=500))
    plt.subplot(2, 2, 3)
    skimage.io.imshow(skimage.filters.gaussian(mp[1], sigma=500))
    plt.subplot(2, 2, 4)
    skimage.io.imshow(skimage.filters.gaussian(mp[2], sigma=500))
    return mp

In [ ]:
from dask.diagnostics import ProgressBar
from skimage import filters
import dask_image.ndfilters


def flatS(fp):
    store = tifffile.imread(fp, aszarr=True)
    dask_array = da.from_zarr(store)
    # print(zc1a.info)
    smooth = dask_image.ndfilters.gaussian_filter(dask_array[:-1], sigma=3)
    mpd = da.max(smooth, axis=0)
    # mpd = np.mean(dask_array[:-1], axis=0)
    # mpd = np.maximum.reduce(dask_array[:-1])
    with ProgressBar():
        mp = mpd.compute()
    plt.subplot(2, 2, 1)
    skimage.io.imshow(mp[1])
    plt.subplot(2, 2, 2)
    skimage.io.imshow(skimage.filters.gaussian(mp[0], sigma=500))
    plt.subplot(2, 2, 3)
    skimage.io.imshow(skimage.filters.gaussian(mp[1], sigma=500))
    plt.subplot(2, 2, 4)
    skimage.io.imshow(skimage.filters.gaussian(mp[2], sigma=500))
    return mp

In [ ]:
flat0S = flatS("/home/dati/dt-clop3/data/210920/flat.tf8")

In [ ]:
store = tifffile.imread("/home/dati/dt-clop3/data/210920/flat.tf8", aszarr=True)
bias = tifffile.imread("/home/dati/dt-clop3/MedianBias20MHz.tif")
dask_array = da.block(da.from_zarr(store))
c0 = dask_array[:, 0]
# smooth = dask_image.ndfilters.gaussian_filter(c0, sigma=60)
smooth = dask_image.ndfilters.gaussian_filter(dask_array, sigma=60)
# smooth = skimage.filters.gaussian(c0, sigma=60)
mpd = da.max(smooth, axis=0)
with ProgressBar():
    mp = mpd.compute()
store.close()

In [ ]:
dask_array gs


In [ ]:
skimage.io.imshow(mp)

In [ ]:
flat0 = flat("/home/dati/dt-clop3/data/210920/flat.tf8")

In [ ]:
skimage.io.imshow(skimage.filters.gaussian(flat0[1], sigma=400), cmap=plt.cm.Set2_r)

In [ ]:
skimage.io.imshow(skimage.filters.gaussian(flat0[1], sigma=400), cmap=plt.cm.Set2_r)

In [ ]:
tifffile.imread("/home/dati/")